In [2]:
import pandas as pd
import math
file = "points.xlsx"
data = pd.read_excel(file)
def compute_player_summaries(data):
    """
    Compute player summaries based on the provided data.
    
    Args:
    - data (pd.DataFrame): The input data containing player details and performance metrics.
    
    Returns:
    - pd.DataFrame: A summary dataframe containing computed metrics for each player.
    """
    # Define aggregation functions for the columns of interest
    aggregations = {
        'Date': 'count',
        'Penalty': 'sum',
        'Friend Referrals': 'sum',
        'Own Goals': 'sum',
        'Goals Conceded': 'sum',
        'Goals': ['mean', 'sum'],
        'Total Points': ['mean', 'sum'],
        'MVP': 'sum',
        'SPL Bonus': 'sum'
    }
    
    # Use groupby with multiple aggregation functions
    player_summary = data.groupby('Player').agg(aggregations)
    
    # Flatten hierarchical columns
    player_summary.columns = ['_'.join(col).strip() for col in player_summary.columns.values]
    
    # Rename columns for clarity
    columns_rename = {
        'Date_count': 'Games Played',
        'Penalty_sum': 'Penalties',
        'Friend Referrals_sum': 'Friend Referrals',
        'Own Goals_sum': 'Own Goals',
        'Goals Conceded_sum': 'Goals Conceded',
        'Goals_mean': 'Goals/Game',
        'Goals_sum': 'Total Goals',
        'Total Points_mean': 'Points/Game',
        'Total Points_sum': 'Total',
        'MVP_sum': 'MVP',
        'SPL Bonus_sum': 'SPL Bonus'
    }
    player_summary.rename(columns=columns_rename, inplace=True)
    
    # Calculate games won
    games_won = data[data['Team'] == data['Winning Team']].groupby('Player').size()
    player_summary['Games Won'] = games_won
    player_summary['Games Won'].fillna(0, inplace=True)
    
    # Calculate win ratio
    player_summary['Win Ratio'] = player_summary['Games Won'] / player_summary['Games Played']
    
    return player_summary

def calculate_cumulative_points_and_rank(data):
    """
    Calculate cumulative points and rank changes for each player.
    
    Args:
    - data (pd.DataFrame): The input data containing player details and performance metrics.
    
    Returns:
    - pd.DataFrame: A dataframe containing the 'Rank Change' for the latest game for each player.
    """
    # Calculate cumulative Total Points for each player after each game
    data['Cumulative Points'] = data.groupby('Player')['Total Points'].cumsum()
    
    # Determine the player's rank based on these cumulative points after each game
    data['Rank'] = data.groupby('Date')['Cumulative Points'].rank(method="first", ascending=False)
    
    # Sort data to ensure we process in chronological order for each player
    data_sorted = data.sort_values(by=['Player', 'Date'])
    
    # Calculate the change in rank between each game for every player
    data_sorted['Rank Change'] = data_sorted.groupby('Player')['Rank'].diff().fillna(0)
    
    # Extract the latest rank change for each player
    latest_rank_change = data_sorted.groupby('Player').apply(lambda x: x.iloc[-1])['Rank Change']
    
    return latest_rank_change

# Integrate functions to generate the final summary
player_summary = compute_player_summaries(data)
latest_rank_change = calculate_cumulative_points_and_rank(data)

# Add 'Rank Change' to the summary
player_summary['Rank Change'] = latest_rank_change

# Fill NaN values with 0
player_summary = player_summary.fillna(0)

# Convert specific columns to integer type
cols_to_int = ['Games Won', 'MVP', 'SPL Bonus', 'Rank Change']
player_summary[cols_to_int] = player_summary[cols_to_int].astype(int)

# Calculate the overall rank based on the 'Total Points'
player_summary['Rank'] = player_summary['Total'].rank(method="min", ascending=False).astype(int)

# Reorder columns to have 'Rank' at the front
column_order = ['Rank'] + [col for col in player_summary if col != 'Rank']
final_summary = player_summary[column_order]

# Sorting the final_summary DataFrame by 'Rank' in ascending order
sorted_summary = final_summary.sort_values(by='Rank')

# Setting option to display all rows
pd.set_option('display.max_rows', None)

# Reordering the columns as specified
desired_column_order = [
    "Rank", "Games Played", "Games Won", "Win Ratio", "Penalties", 
    "Friend Referrals", "Own Goals", "Goals Conceded", "MVP", "SPL Bonus",
    "Goals/Game", "Total Goals", "Points/Game", 
    "Total", "Rank Change"
]

for col in sorted_summary.columns:
    if sorted_summary[col].dtype == 'float64':
        sorted_summary[col] = sorted_summary[col].round(2)
sorted_summary['Win Ratio'] = (sorted_summary['Win Ratio'] * 100).round(0).astype(int).astype(str) + '%'
sorted_summary = sorted_summary[desired_column_order].dropna()
sorted_summary = sorted_summary.reset_index()
sorted_summary


,Player,Rank,Games Played,Games Won,Win Ratio,Penalties,Friend Referrals,Own Goals,Goals Conceded,MVP,SPL Bonus,Goals/Game,Total Goals,Points/Game,Total,Rank Change
0,Daniele (Francesco),1,17,12,71%,0,1,0,108,0,0,3.59,61,14.47,246,0
1,Luca Stoppi,2,23,8,35%,1,4,2,166,1,1,0.78,18,10.65,245,1
2,Andrea Limonta,3,22,11,50%,0,2,1,154,0,1,0.14,3,10.50,231,0
3,Cameron McAinsh,4,22,9,41%,0,1,0,181,0,0,1.82,40,10.32,227,-1
4,Damiano Barbanera,5,21,7,33%,0,2,0,161,0,1,2.29,48,10.57,222,-2
5,Giovanni Aiello,6,21,11,52%,0,0,0,147,0,2,0.57,12,10.14,213,2
6,Sergio,7,17,9,53%,0,2,0,106,2,0,0.71,12,10.76,183,2
7,Federico Paolucci,8,19,9,47%,0,0,0,140,1,1,0.26,5,9.47,180,-2
8,Mazzu,9,14,4,29%,0,3,1,110,1,2,3.29,46,12.64,177,-2
9,Roby (Cri),10,16,5,31%,0,0,0,130,0,2,0.69,11,9.81,157,0


In [3]:
# Save the games_df DataFrame as an Excel file
sorted_summary_excel = "sorted_summary.xlsx"
sorted_summary.to_excel(sorted_summary_excel, index=False)

In [4]:
def df_to_html_updated(df):
    # Style the specific columns to be bold
    def bold_columns(series):
        col_name = series.name
        return series.map(lambda val: "font-weight: bold" if col_name in ["Player", "Rank", "Total"] else "")

    styled_df = df.style.apply(bold_columns, axis=0, subset=["Player", "Rank", "Total"])

    # Convert styled dataframe to HTML table without the default index and header tags
    table_content = styled_df.to_html(index=False, header=True)
    
    # Basic HTML structure with a placeholder for the table content
    html = f"""
       <html>
        <head>
            <style>
                body {{
                    font-family: Arial, sans-serif;
                    margin: 0;
                    padding: 0;
                }}
                table {{
                    width: 100%;
                    border-collapse: collapse;
                    margin: 25px 0;
                    font-size: 15px;
                    text-align: left;
                }}
                th, td {{
                    padding: 10px;
                    border-bottom: 1px solid #ddd;
                }}
                th {{
                    white-space: normal;
                }}
                tr:nth-child(1) {{
                    background-color: gold;
                }}
                tr:nth-child(2) {{
                    background-color: silver;
                }}
                tr:nth-child(3) {{
                    background-color: #cd7f32; /* Bronze color */
                }}
                tr:nth-child(even):not(:nth-child(-n+3)) {{
                    background-color: #f2f2f2;
                }}
                tr:nth-child(odd):not(:nth-child(-n+3)) {{
                    background-color: #ffffff;
                }}
                tr:hover {{
                    background-color: #f5f5f5;
                }}
                th {{
                    background-color: #333;
                    color: white;
                    text-align: center;
                }}
                td {{
                    text-align: center;
                }}
                td:first-child {{
                    text-align: left;
                }}
                @media (max-width: 768px) {{
                    th, td {{
                        font-size: 13px;
                    }}
                }}
            </style>
        </head>
        <body>
            {table_content}
        </body>
    </html>
    """
    
    return html

# Generate the HTML
html_content = df_to_html_updated(sorted_summary)

# Save the HTML content to a file
html_file_path = "players_stats.html"
with open(html_file_path, "w") as f:
    f.write(html_content)

html_file_path


'players_stats.html'

In [5]:
import pandas as pd
from bs4 import BeautifulSoup

# Read the HTML content from the file
with open("players_stats.html", 'r') as file:
    html_content = file.read()

# Parse the HTML content using BeautifulSoup
soup = BeautifulSoup(html_content, 'html.parser')

# Extract the table from the parsed HTML
table = soup.find('table')

# Convert the HTML table to a pandas DataFrame
df = pd.read_html(str(table))[0]

# Round all numeric values to 2 decimal points
df = df.round(2)

# Remove the column before "Player"
df = df.drop(columns=["Unnamed: 0"])

# Convert the DataFrame back to an HTML table
html_table = df.to_html(index=False, classes="players-stats")

# Extract the styling from the original HTML
styles = soup.find('style').string

# Create a new HTML structure combining the styles and the new table
new_html_content = f"""
<html>
<head>
    <style>
        {styles}
    </style>
</head>
<body>
    {html_table}
</body>
</html>
"""

# Save the modified HTML content to a new file
with open("players_stats_new.html", 'w') as file:
    file.write(new_html_content)
